In [1]:
# Load AIEEE Data

In [1]:
!nvidia-smi

Fri Apr  1 09:39:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   30C    P0    34W / 250W |   2608MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

In [3]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [4]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [5]:
er_df = pd.read_csv("Models/Data/ERData/er_data_with_majority_label.csv")
er_df = er_df.dropna()

In [6]:
er_df.shape

(7149488, 3)

Load Models

In [7]:
#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [8]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [9]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [10]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)

In [ ]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)

In [ ]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_BERT)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


Predict Gender

In [12]:
from transformers import pipeline

In [13]:
from sklearn.metrics import classification_report

# print(classification_report(true, labels, digits = 4))

In [14]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
        with open("Models/ERData/er_data_aieee_bert_test1.txt", 'a') as f:
            f.write(f"{i}\n")
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    report = classification_report(true, labels, digits = 4)
    print(classification_report(true, labels, digits = 4))
    with open("Models/ERData/er_data_aieee_bert_test1.txt", 'a') as f:
            f.write(f"{report}\n")
    return report

In [15]:
er_df.shape

(7149488, 3)

In [ ]:
report1 = print_results(aieee_model, tokenizer, er_df[:2000000])

In [ ]:
report2 = print_results(aieee_model, tokenizer, er_df[2000000:4000000])

In [ ]:
report3 = print_results(aieee_model, tokenizer, er_df[4000000:6000000])

In [ ]:
report4 = print_results(aieee_model, tokenizer, er_df[6000000:])